In [1]:
import os
import pandas as pd
import numpy as np
import pyhomogeneity as hg
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
from scipy.stats import skew, t
%config Completer.use_jedi = False #for auto complete code 
def transform_precipitation_data(data):
    # Replace commas with dots for decimal conversion
    data = data.replace(',', '.', regex=True)

    # Convert the DataFrame to numeric, handling errors
    for column in data.columns[2:]:
        data[column] = pd.to_numeric(data[column], errors='coerce')

    # Rename columns to match the expected names
    data.rename(columns={'Ano': 'Año', 'dia': 'Día'}, inplace=True)

    # Melt the DataFrame
    df_melted = pd.melt(data, id_vars=['Año', 'Día'], value_vars=['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Set', 'Oct', 'Nov', 'Dic'],
                        var_name='Mes', value_name='Precipitación')

    # Map Spanish month abbreviations to numbers
    month_mapping = {
        'Ene': 1, 'Feb': 2, 'Mar': 3, 'Abr': 4, 'May': 5, 'Jun': 6,
        'Jul': 7, 'Ago': 8, 'Set': 9, 'Oct': 10, 'Nov': 11, 'Dic': 12
    }
    df_melted['Mes'] = df_melted['Mes'].map(month_mapping)

    # Ensure the correct data types for the columns
    df_melted['Año'] = df_melted['Año'].astype(str)
    df_melted['Mes'] = df_melted['Mes'].astype(str)
    df_melted['Día'] = df_melted['Día'].astype(str)

    # Function to convert to datetime with error handling
    def convert_to_datetime(row):
        try:
            return pd.to_datetime(f"{row['Año']}-{row['Mes']}-{row['Día']}")
        except ValueError:
            return pd.NaT  # Return NaT for invalid dates

    # Apply the function to create 'Fecha' column
    df_melted['Fecha'] = df_melted.apply(convert_to_datetime, axis=1)

    # Select the desired columns
    df_final = df_melted[df_melted['Fecha'].notna()][['Fecha', 'Precipitación']] # Drop rows with NaT values

    return df_final

In [2]:
files = ['../data/precp/chusis.csv','../data/precp/chalaco.csv','../data/precp/huarmaca.csv','../data/precp/huancabamba.csv','../data/precp/miraflores.csv']

resultados = []
for file in files:
    obs=pd.read_csv(file,sep=';')
    df = transform_precipitation_data(obs)
    df.set_index('Fecha',inplace=True)
    # Sort the DataFrame by the index (Fecha)
    df.sort_index(inplace=True)
    # Eliminate duplicate rows based on the index (Fecha)
    df = df[~df.index.duplicated(keep='first')]
    resultados.append(df)
    

In [11]:
Estaciones = ['chusis','chalaco','huamarca','huancabamba','miraflores']
# Concatenate them horizontally
merged_df = pd.concat(resultados, axis=1)

In [14]:
merged_df.columns=Estaciones

In [15]:
merged_df

,chusis,chalaco,huamarca,huancabamba,miraflores
Fecha,,,,,
1980-01-01,0.0,0.0,0.4,2.1,0.0
1980-01-02,0.0,0.0,0.0,0.0,0.0
1980-01-03,0.0,0.0,0.0,1.5,0.0
1980-01-04,0.0,0.0,0.0,0.0,0.0
1980-01-05,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...
2017-12-27,NaN,NaN,2.8,NaN,0.0
2017-12-28,NaN,NaN,12.2,NaN,0.2
2017-12-29,NaN,NaN,2.4,NaN,0.0


In [16]:
merged_df.to_csv('../data/precp/gauge.csv',index=True)